In [17]:
!pip install unidecode


In [18]:
import pandas as pd
from unidecode import unidecode
import string

In [19]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
csv_file = "dataQuestionnaire.csv"
data = pd.read_csv(csv_file)
texte_colonnes = ["Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)",
                 "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)",
                 "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)",
                 "Pouvez-vous expliquer vos réponses merci ..."]

In [22]:
chaines_par_colonne = []

for colonne in texte_colonnes:
    chaine = ' '.join(str(value) for value in data[colonne] if pd.notna(value))
    chaines_par_colonne.append(chaine)

def preprocess_text(text):
    text_lower = text.lower()
    return text_lower

chaines_par_colonne_preprocessed = []

for colonne in texte_colonnes:
    chaine = ' '.join(str(value) for value in data[colonne] if pd.notna(value))
    chaine_preprocessed = preprocess_text(chaine)
    chaines_par_colonne_preprocessed.append(chaine_preprocessed)

In [23]:
total_text = chaines_par_colonne[0] + chaines_par_colonne[1] + chaines_par_colonne[2] + chaines_par_colonne[3]

In [24]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
# Créer un Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatizer_texte(texte):
    mots = nltk.word_tokenize(texte, language='french')
    lemmatizer = WordNetLemmatizer()
    lemmes = [lemmatizer.lemmatize(mot, pos=wordnet.VERB) for mot in mots]
    texte_lemmatise = ' '.join(lemmes)

    return texte_lemmatise

texte_lemmatise = lemmatizer_texte(total_text)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

french_stop_words = []

In [26]:
vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words=french_stop_words)
X = vectorizer.fit_transform([texte_lemmatise])

feature_names = vectorizer.get_feature_names_out()

occurrences = X.sum(axis=0)

ngrams_freq = [(feature_names[col], occurrences[0, col]) for col in range(occurrences.shape[1])]
ngrams_freq.sort(key=lambda x: x[1], reverse=True)

 #reponses par rapport a une question du questionnaire qui a ete mal comprise, d'ou les nombreux commentaires
mots_a_exclure = ["questionnaire", "question", "inscrits",
                  #pour les ngrams avec 3 ou moins mots
                  "la", "le", "l", "les", "des", "qui", "quoi", "des", "un", "une"
                   #pour les ngrams avec 2 mots ou moins
                  , "et", "dans", "de", "ce", "je", "qu", "suis", "du", "en", "par", "mais", "au", "mon", "notre", "se", "ai", "est",
                  "nous", "moi", "on", "me", "avec", "soi", "ma", "mon", "nos", "eux", "ne", "es"
]
for rank, (ngram, freq) in enumerate(ngrams_freq):
  #if freq <= 4 and len(ngram) > 2 and not any(mot in ngram for mot in mots_a_exclure) :
  #if freq > 1 and not any(mot in ngram for mot in mots_a_exclure) : # Pour les ngrams contenant plus de 4 mots
    if freq > 10 and not any(mot in ngram for mot in mots_a_exclure) : #and "réforme" in ngram :
        print(f"{rank + 1}. {ngram} : {freq} occurrences")


9. pour : 643 occurrences
10. pas : 596 occurrences
13. recherche : 520 occurrences
22. temps : 310 occurrences
28. métier : 267 occurrences
30. sur : 253 occurrences
36. tout : 184 occurrences
37. très : 177 occurrences
40. ou : 156 occurrences
41. charge : 149 occurrences
43. trop : 136 occurrences
46. étudiants : 130 occurrences
50. être : 122 occurrences
54. toujours : 109 occurrences
56. peu : 107 occurrences
58. sans : 103 occurrences
59. si : 102 occurrences
62. avoir : 98 occurrences
67. carrière : 81 occurrences
68. chercheurs : 79 occurrences
70. nombre : 79 occurrences
73. ans : 77 occurrences
79. chercheur : 71 occurrences
91. année : 61 occurrences
93. ça : 61 occurrences
96. peut : 60 occurrences
100. où : 58 occurrences
103. car : 55 occurrences
105. post : 55 occurrences
107. cours : 52 occurrences
120. activité : 45 occurrences
122. privé : 44 occurrences
124. tous : 44 occurrences
131. mcf : 41 occurrences
132. surtout : 41 occurrences
133. été : 41 occurrences
134. p

In [27]:
resultats = [
    ["aime toujours faire de la recherche", 15],
    ["de plus en plus de contraintes", 3],
    ["avait un peu moins de boulets qui ne pensent qu leur recherche dans nos universités", 2],
    ["distance en temps pour transiter de votre lieu enseignement celui de la recherche", 2],
    ["il manque au moins post ec dans les unités où interviens", 2],
    ["manque au moins post ec dans les unités", 2],
    ["décidé de ne plus répondre aux appels projets", 2],
    ###### 7 mots
    ["de plus en plus de tâches administratives", 3],
    ####### 6 mots
    [" de plus en plus de contraintes", 3],
    ["la réforme du but gravement dégradé", 3],
    ["la difficulté de trouver un poste", 2],
    ########### 5 mots
    ["de plus en plus difficiles", 2],
    ["de plus en plus dur", 2],
    ["de plus en plus grande", 2],
    ["de plus en plus importantes", 2],
    ["de plus en plus lourdes", 2],
    ["difficile de conseiller ce métier", 2],
    ["difficile de trouver des candidats", 2],
    ["difficulté de trouver un poste", 2],
    ["des avantages et des inconvénients", 2],
    ["des raisons de charge enseignement", 2],
    ["plus du tout le même métier", 2],
    ["dégradation des condition de travail", 2],
    ["dégradé les condition de travail", 2],
    ["les condition de travail dégradées", 2],
    ["intéressant mais les charge administratives", 2],
    ["la dégradation des condition exercice", 2],
    ["la quantité de charge administratives", 2],
    ######### 4 mots
    ["trop de tâches administratives", 2],
    ["trop de charge administratives", 2],
    ["le manque de personnel administratif", 2],
    ["le métier est intéressant", 2],
    ["le métier est très intéressant", 2],
    ["les charge enseignement et administratives", 2],
    ["la réforme du but", 4],
    ["le manque de personnel", 4],
    ["le manque de reconnaissance", 4],
    ["le manque de temps", 4],
    ["administration de la recherche", 3],
    ["en fin de carrière", 3],
    ["plus de tâches administratives", 3],
    ["une perte de sens", 3],
    ["une perte de temps", 3],
    ["aucune perspective de promotion", 2],
    ["aucune reconnaissance de investissement", 2],
    ["beaucoup aimé mon métier", 2],
    ["charge administratives et enseignement", 2],
    ["charge enseignement et administration", 2],
    ["charge enseignement et administratives", 2],
    ["charge pédagogiques et administratives", 2],
    ["quantité de charge administratives", 2],
    ["tâches enseignement et administration", 2],
    ["responsabilités administratives et pédagogiques", 2],
    ["condition de travail dégradées", 2],
    ["conditions de travail dégradées", 2],
    ["les condition se dégradent", 2],
    ["de nombreuses tâches administratives", 2],
    ["détriment de ma recherche", 2],
    ["je conseille ce métier", 2],
    ["jamais demandé de prime", 2],
    ["la baisse des moyens", 2],
    ["la quantité de charge", 2],
    ["la quantité de travail", 2],
    ["la reconnaissance de investissement", 2],
    ["la surcharge de travail", 2],
    ["moyens humains et financier", 2],
    ["pas assez de moyens", 2],
    ["pas assez de post", 2],
    ["pas beaucoup de sens", 2],
    ["pas du tout reconnu", 2],
    ["perdre beaucoup de temps", 2],
    ["un temps de recherche",2],
    ["un travail de fond", 2],
    ["une charge de travail", 2],
    ["une prestation de service", 2],
    ["une prestation de service", 2],
    ["équipes techniques et administratives", 2],
    ["difficile obtenir un poste", 2],
    ############# 3 mots
    ["étrange et plein hypocrisie", 2],
    ["activité de recherche", 22],
    ["début de carrière", 15],
    ["manque de moyens", 12],
    ["perte de sens", 6],
    ["tout années confondues", 6],
    ["travaux de recherche", 6],
    ["aime mon métier", 5],
    ["docteur en informatique", 5],
    ["enseignement recherche administration", 5],
    ["fin de carrière", 5],
    ["financement par projet", 5],
    ["métier est intéressant", 5],
    ["recherche aime toujours", 5],
    ["recherche de financement", 5],
    ["trop de charge", 5],
    ["adore mon métier", 4],
    ["avancement de carrière", 4],
    ["distance en temps", 4],
    ["manque de post", 4],
    ["manque de reconnaissance", 4],
    ["moins de temps", 4],
    ["métier est passionnant", 4],
    ["peu de recherche", 4],
    ["peu de reconnaissance", 4],
    ["projets de recherche", 4],
    ["reconnaissance de investissement",4],
    ["réforme du but", 4],
    ["absurde et chaotique",],
    ["administration de enseignement", 3],
    ["administratives et enseignement", 3],
    ["administratives et pédagogiques", 3],
    ["attractivité du métier", 3],
    ["beaucoup de charge", 3],
    ["but gravement dégradé", 3],
    ["manque de considération", 3],
    ["partir dans industrie", 3],
    ["pas de promotion", 3],
    ["pas de sens", 3],
    ["perspective de promotion", 3],
    ["contraintes administratives croissantes",2],
    ["nombreuses tâches administratives", 2],
    ["parcours du combattant", 2],
    ["partir en mission", 2],
    ["pas assez enseignants", 2],
    ["pas assez payé", 2],
    ["pas de cours", 2],
    ["pas de moyens", 2],
    ["perdre du temps", 2],
    ["perte de considération", 2],
    ["peu de post", 2],
    ["peu de retours", 2],
    ["privé en informatique", 2],
    ["progression de carrière", 2],
    ["projet de recherche", 2],
    ["quantité de charge", 2],
    ["recherche de crédits", 2],
    ["recherche de financements", 2],
    ["réduit mon activité", 2],
    ["suis très content", 2],
    [" sujet de recherche", 2],
    ["sujet de thèse", 2],
    ["système de financement", 2],
    ["temps de transport", 2],
    ["toujours ma passion", 2],
    ["trop de tâches", 2],
    ["trop heures enseignemen", 2],
    ["trop heures sup", 2],
    [" très beau métier", 2],
    ["vide de sens", 2],
    ######### 2 mots
    ["charge administrative", 12],
    ["burn out", 7],
    ["temps recherche", 4],
    ["coûts cachés", 3],
    ["métier étrange", 3],
    ["point positifs", 3],
    ["support administratif", 3],
    ["trop administratif", 3],
    ["pas attirant", 2],
    ["pas attractif", 2],
    ["pas candidater", 2],
    ["peu attrayant", 2],
    ########## 1 mot
    ["carrière", 81],
    ["chercheurs", 79],
    ["recherche", 520],
    ["métier", 267],
    ["charge", 149],
    ["étudiants", 130],
    ["chercheur", 71],
    ["activité", 45],
    ["mcf", 41],
    ["doctorants", 39],
    ["administratif", 38],
    ["administrative", 38],
    ["activités", 29],
    ["administratifs", 16],
    ["attirant", 16],
    ["doctorant", 16],
    ["surcharge", 14],
    ["covid", 12]
]

In [28]:
negatif = ['pas', 'charge', 'trop', 'peu', 'privé', 'administratif', 'administrative', 'perte', 'administratifs', 'but',
           'décharge', 'surcharge', 'covid', 'dégradé', 'perdre', 'doute', 'arrêter', 'admin', 'burn', 'cdd', 'out', 'refus',
           'empiète', 'fatigue', 'fou', 'cher', 'démotive', 'empêche', 'négatif', 'épuisant', 'adm', 'burnout', 'copinage',
           'décourageant', 'décrocher', 'frein', 'frustrant', 'handicap', 'misère', 'aucune', 'aucun', 'poids', 'sacrifier',
           'smic', 'soucis', 'souffre', 'transport', 'décourage', 'dégradée', 'forcé', 'fuite', 'guerre', 'hypocrisie',
           'insuffisant', 'négatifs', 'souci', 'tue', 'catastrophe', 'décourager', 'déplacement', 'déplacer', 'réforme', 'ni']

neutre = ['temps', 'métier', 'étudiants', 'si', 'carrière', 'chercheurs', 'nombre', 'ans', 'chercheur', 'année', 'post', 'cours',
          'activité', 'mcf', 'poste', 'doctorants', 'pouvoir', 'rapport', 'compte', 'vie', 'passé', 'trouver', 'activités', 'iut',
         'support', 'avant', 'après', 'dossier', 'compris', 'dire', 'fois', 'forte', 'important', 'choix', 'docteur', 'début', 'juste',
         'voir', 'étant', 'gérer', 'fin', 'futur', 'intérêt', 'ufr', 'doctorant', 'dossiers', 'candidats', 'arrêté', 'pédagogie', 'cnrs',
          'financier', 'aspect', 'cnu', 'suivi', 'étudiant', 'ingénieur', 'métiers', 'offre', 'ingénieurs', 'sup', 'état', 'assurer',
          'directeur', 'doctorat', 'recruter', 'supérieur', 'société', 'prof', 'avis', 'effectif', 'effectifs', 'statut', 'candidat',
          'chef', 'santé', 'structure', 'thésards', 'tâche', 'impact', 'iuf', 'ministère', 'perspective', 'stage', 'stratégie']

positif = ['recherche', 'sur', 'sans', 'ripec', 'docteurs', 'attirant', 'étranger', 'anr', 'hdr', 'attractif', 'attractivité',
           'ripec3', 'inria', 'cadre', 'ins2i', 'motivés', 'avantage', ' apprécie', 'attirer', 'attractifs', 'gratifiant', 'motivant',
           'réussi', 'succès', 'suffisant', 'bénéficier', 'cdi', 'positifs', 'ras', 'réussir', 'réussite', 'rêve', 'bénéficie',
           'motivé', 'mérite', 'positif', 'rechercher']

mots_pas_sur = ['ripec', 'anr', 'hdr', 'cnrs', 'cnu', 'ripec3', 'sit', 'inria', 'starynkevitch', 'ins2i', 'refpersys']
